In [199]:
from csv import reader
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

In [200]:
entity_size_mean = pd.read_parquet('../results/entity.parquet')
with open('../results/category.pickle', 'rb') as handle:
    category = pickle.load(handle)
    
entity_size_mean = entity_size_mean.groupby('movieId').agg({'rating': [np.size, np.mean]})
entity_size_mean.head()

rating          
            size      mean
movieId                   
1        57309.0  3.893708
2        24228.0  3.251527
3        11804.0  3.142028
4         2523.0  2.853547
5        11714.0  3.058434

# Normalização (0-1)

In [201]:
entityRatings = pd.DataFrame(entity_size_mean['rating']['size'])
entityNormalizedNumRatings = entityRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
entityNormalizedNumRatings.head()

,size
movieId,
1,0.703252
2,0.297300
3,0.144840
4,0.030949
5,0.143735


In [ ]:
movieDict = {}
entity = pd.read_parquet('../results/entity.parquet')[['title', 'genres', 'movieId']]
for index, row in entity.iterrows():
    movieID = int(row['movieId'])
    name = category['title'][row['title']]
    genres = category['genres'][row['genres']].split('|')
    movieDict[movieID] = (name, np.array(list(genres)), entityNormalizedNumRatings.loc[movieID].get('size'), entity_size_mean.loc[movieID].rating.get('mean'))

In [225]:
movieDict

{}

In [227]:
np.array(list(category['genres'][1160].split('|')))

array(['Comedy', 'Crime', 'Drama', 'Thriller'], dtype='<U8')

In [181]:
entity = pd.read_parquet('../results/entity.parquet')[['title', 'genres', 'movieId']]
entity = entity.drop_duplicates()
entity.genres = entity.genres.map(category['genres'])

In [182]:
entity = entity.assign(genres=entity['genres'].str.split('|')).explode('genres')

In [183]:
entity = pd.concat([entity.drop('genres', 1), pd.get_dummies(entity.genres).mul(1)], axis=1).sum(level=0)

In [184]:
entity.set_index('movieId', inplace=True)

In [185]:
entity = entity.merge(entityNormalizedNumRatings, how='left', on=['movieId'])

In [186]:
entity = entity.merge(entity_size_mean['rating']['mean'], how='left', on=['movieId'])

In [188]:
entity = entity.reset_index()

In [190]:
def euclidean_distance(a, b):
    distance = 0.0
    for i in range(len(a)-1):
        distance += (a[i] - b[i])**2
    return sqrt(distance)

def ComputeDistance(genresA, genresB):
    #genreDistance = spatial.distance.cosine(genresA, genresB)
    euclideanDistance = euclidean_distance(genresA, genresB)
    popularityA = genresA["size"]
    popularityB = genresB["size"]
    popularityDistance = abs(popularityA - popularityB)
    return euclideanDistance + popularityDistance

ComputeDistance(entity.iloc[1], entity.iloc[2])

1.4196479421158235

In [191]:
entity[entity['title']==39812]

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,size,mean
21512,52066,39812,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [196]:
import operator

def getNeighbors(movieID, K):
    distances = []
    for movie in entity.index:
        if (movie != movieID):
            dist = ComputeDistance(entity.iloc[movieID], entity.iloc[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K = 10
avgRating = 0

In [197]:
print(entity.iloc[1], '\n')

movieId                 306.000000
title                 56015.000000
(no genres listed)        0.000000
Action                    0.000000
Adventure                 0.000000
Animation                 0.000000
Children                  0.000000
Comedy                    0.000000
Crime                     0.000000
Documentary               0.000000
Drama                     1.000000
Fantasy                   0.000000
Film-Noir                 0.000000
Horror                    0.000000
IMAX                      0.000000
Musical                   0.000000
Mystery                   0.000000
Romance                   0.000000
Sci-Fi                    0.000000
Thriller                  0.000000
War                       0.000000
Western                   0.000000
size                      0.086600
mean                      4.072967
Name: 1, dtype: float64 



In [198]:
neighbors = getNeighbors(1, K) # Toy Story (1995)
for neighbor in neighbors:
    avgRating += entity.iloc[neighbor][3]
    print (entity.iloc[neighbor][0] + " " + str(entity.iloc[neighbor][3]))

avgRating /= K

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

In [ ]:
movieDict = {}
temp = ''
for line in entity:
    movieID = int(fields[0])
    name = fields[1]
    genres = fields[5:25]
    genres = map(int, genres)
    movieDict[movieID] = (name, np.array(list(genres)), movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))

In [4]:
entity.values.tolist()

[[296, 1, 5.0, 1147880044, None, 39812, 'Comedy|Crime|Drama|Thriller'],
 [296, 3, 5.0, 1439474476, None, 39812, 'Comedy|Crime|Drama|Thriller'],
 [296, 4, 4.0, 1573938898, None, 39812, 'Comedy|Crime|Drama|Thriller'],
 [296, 5, 4.0, 830786155, None, 39812, 'Comedy|Crime|Drama|Thriller'],
 [296, 7, 4.0, 835444730, None, 39812, 'Comedy|Crime|Drama|Thriller'],
 [296, 8, 5.0, 890489713, None, 39812, 'Comedy|Crime|Drama|Thriller'],
 [296, 10, 4.5, 1227571308, None, 39812, 'Comedy|Crime|Drama|Thriller'],
 [296, 12, 5.0, 1119354604, None, 39812, 'Comedy|Crime|Drama|Thriller'],
 [296, 13, 5.0, 1238029599, None, 39812, 'Comedy|Crime|Drama|Thriller'],
 [296, 14, 5.0, 1506208897, None, 39812, 'Comedy|Crime|Drama|Thriller'],
 [296, 15, 5.0, 946013314, None, 39812, 'Comedy|Crime|Drama|Thriller'],
 [296, 18, 3.0, 1108273426, None, 39812, 'Comedy|Crime|Drama|Thriller'],
 [296, 19, 3.5, 1466674637, None, 39812, 'Comedy|Crime|Drama|Thriller'],
 [296, 22, 3.0, 832862128, None, 39812, 'Comedy|Crime|Drama|T

In [11]:
np.array(list(genres))

NameError: name 'genres' is not defined

In [12]:
category['genres']

{0: '(no genres listed)',
 1: 'Action',
 2: 'Action|Adventure',
 3: 'Action|Adventure|Animation',
 4: 'Action|Adventure|Animation|Children',
 5: 'Action|Adventure|Animation|Children|Comedy',
 6: 'Action|Adventure|Animation|Children|Comedy|Fantasy',
 7: 'Action|Adventure|Animation|Children|Comedy|IMAX',
 8: 'Action|Adventure|Animation|Children|Comedy|Romance',
 9: 'Action|Adventure|Animation|Children|Comedy|Sci-Fi',
 10: 'Action|Adventure|Animation|Children|Comedy|Sci-Fi|IMAX',
 11: 'Action|Adventure|Animation|Children|Comedy|Western',
 12: 'Action|Adventure|Animation|Children|Crime|Mystery',
 13: 'Action|Adventure|Animation|Children|Drama',
 14: 'Action|Adventure|Animation|Children|Fantasy',
 15: 'Action|Adventure|Animation|Children|Fantasy|Sci-Fi',
 16: 'Action|Adventure|Animation|Children|Mystery',
 17: 'Action|Adventure|Animation|Children|Sci-Fi',
 18: 'Action|Adventure|Animation|Comedy',
 19: 'Action|Adventure|Animation|Comedy|Crime|Mystery',
 20: 'Action|Adventure|Animation|Comedy

In [145]:
entity = entity.drop('timestamp_rating', axis=1)
entity = entity.drop('userId', axis=1)

In [148]:
movieProperties = entity.groupby('movieId').agg({'rating': [np.size, np.mean]})
movieProperties.head()

rating          
            size      mean
movieId                   
1        57309.0  3.893708
2        24228.0  3.251527
3        11804.0  3.142028
4         2523.0  2.853547
5        11714.0  3.058434

In [149]:
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()

,size
movieId,
1,0.703252
2,0.297300
3,0.144840
4,0.030949
5,0.143735


In [ ]:
movieDict = {}

entity = pd.read_parquet('../results/entity.parquet')[['movieId', 'rating']]


temp = ''
for line in entity.index:
    movieID = int(fields[0])
    name = fields[1]
    genres = fields[5:25]
    genres = map(int, genres)
    movieDict[movieID] = (name, np.array(list(genres)), movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))

In [120]:
entity = entity.groupby(['title', 'movieId', 'genres'], as_index=False).mean()

In [72]:
X = entity.drop('movieId', axis=1)
y = entity.movieId

In [ ]:
# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
    distances = list()
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

In [130]:
entity[0:10]

['title', 'movieId', 'genres', 'rating']

# Define a function to calculate distance between two points# 

In [140]:
def euclidean_distance(a, b):
    distance = 0.0
    for i in range(len(a)-1):
        distance += (a[i] - b[i])**2
    return sqrt(distance)


# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train.index:
		dist = euclidean_distance(test_row, train.iloc[train_row])
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors

In [141]:
X.iloc[0]

title        0.0
genres    1399.0
rating       3.0
Name: 0, dtype: float64

In [142]:
neighbors = get_neighbors(X, [1, 1599, 2.896648], 3)
for neighbor in neighbors:
	print(neighbor)

1
5
3


In [132]:
X.iloc[i]

title     62324.0
genres        0.0
rating        4.0
Name: 59046, dtype: float64

In [90]:
%%time
test_pt = [ 1, 1599, 2.896648]

# Calculate distance between test_pt and all points in X
distances = []
for i in X.index:
    distances.append(euclidean_distance(test_pt, X.iloc[i]))
    

CPU times: user 9.24 s, sys: 35.8 ms, total: 9.28 s
Wall time: 9.3 s


In [91]:
df_dists = pd.DataFrame(data=distances, index=X.index, columns=['dist'])

,dist
0,200.002500
1,0.000000
2,409.001222
3,28.071338
4,201.022387


In [92]:
df_nn = df_dists.sort_values(by=['dist'], axis=0)[:10]
df_nn

,dist
1,0.000000
5,8.062258
3,28.071338
8,28.861739
13,40.804412


In [93]:
from collections import Counter

counter = Counter(y[df_nn.index])
counter.most_common()[0][0]

51372

In [47]:
a[d]

[100, 200]

In [89]:
X.iloc[1]

title        1.000000
genres    1599.000000
rating       2.896648
Name: 1, dtype: float64

In [88]:
entity.iloc[1]

title          1.000000
movieId    51372.000000
genres      1599.000000
rating         2.896648
Name: 1, dtype: float64

In [35]:
X.at[0, 'tag'] = [100, 200]

In [37]:
X.at[1, 'tag'] = [300, 400]

In [50]:
X.iloc[1]

userId                       3
rating                       5
timestamp_rating    1439474476
tag                 [300, 400]
title                    39812
genres                    1160
Name: 1, dtype: object

In [124]:
from collections import Counter
import math

def knn(data, query, k, distance_fn, choice_fn):
    neighbor_distances_and_indices = []
    
    # 3. For each example in the data
    for index, example in enumerate(data):
        # 3.1 Calculate the distance between the query example and the current
        # example from the data.
        distance = distance_fn(example[:-1], query)
        
        # 3.2 Add the distance and the index of the example to an ordered collection
        neighbor_distances_and_indices.append((distance, index))
    
    # 4. Sort the ordered collection of distances and indices from
    # smallest to largest (in ascending order) by the distances
    sorted_neighbor_distances_and_indices = sorted(neighbor_distances_and_indices)
    
    # 5. Pick the first K entries from the sorted collection
    k_nearest_distances_and_indices = sorted_neighbor_distances_and_indices[:k]
    
    # 6. Get the labels of the selected K entries
    k_nearest_labels = [data[i][-1] for distance, i in k_nearest_distances_and_indices]

    # 7. If regression (choice_fn = mean), return the average of the K labels
    # 8. If classification (choice_fn = mode), return the mode of the K labels
    return k_nearest_distances_and_indices , choice_fn(k_nearest_labels)

def mean(labels):
    return sum(labels) / len(labels)

def mode(labels):
    return Counter(labels).most_common(1)[0][0]

def euclidean_distance(point1, point2):
    sum_squared_distance = 0
    for i in range(len(point1)):
            sum_squared_distance += math.pow(point1[i] - point2[i], 2)
    return math.sqrt(sum_squared_distance)

In [127]:
recommendation_indices, _ = knn(
        entity.drop('movieId', axis=1), 
        [1, 1599, 2.896648], 
        k=1,
        distance_fn=euclidean_distance, choice_fn=lambda x: None
    )


t
1
i
1599
t
2.896648
l


IndexError: list index out of range

In [122]:
entity.drop('movieId', axis=1)

,title,genres,rating
0,0,1399,3.000000
1,1,1599,2.896648
2,2,1190,2.111111
3,3,1571,3.750000
4,4,1398,3.666667
...,...,...,...
59042,62320,1524,3.250000
59043,62321,1287,3.250000
59044,62322,891,3.166667
59045,62323,1571,2.375000


In [110]:
def recommend_movies(raw_movies_data, movie_query, k_recommendations):
    raw_movies_data = []
    with open('movies_recommendation_data.csv', 'r') as md:
        # Discard the first line (headings)
        next(md)

        # Read the data into memory
        for line in md.readlines():
            data_row = line.strip().split(',')
            raw_movies_data.append(data_row)

    # Prepare the data for use in the knn algorithm by picking
    # the relevant columns and converting the numeric columns
    # to numbers since they were read in as strings
    movies_recommendation_data = []
    for row in raw_movies_data:
        data_row = list(map(float, row[2:]))
        movies_recommendation_data.append(data_row)

    # Use the KNN algorithm to get the 5 movies that are most
    # similar to The Post.
    recommendation_indices, _ = knn(
        movies_recommendation_data, movie_query, k=k_recommendations,
        distance_fn=euclidean_distance, choice_fn=lambda x: None
    )

    movie_recommendations = []
    for _, index in recommendation_indices:
        movie_recommendations.append(raw_movies_data[index])

    return movie_recommendations

NameError: name 'operator' is not defined

In [105]:
entity

,title,movieId,genres,rating
0,0,208297,1399,3.000000
1,1,51372,1599,2.896648
2,2,136604,1190,2.111111
3,3,183901,1571,3.750000
4,4,195955,1398,3.666667
...,...,...,...,...
59042,62320,159678,1524,3.250000
59043,62321,132604,1287,3.250000
59044,62322,132319,891,3.166667
59045,62323,130640,1571,2.375000


In [102]:
def train_test_split(dataset, test_size=0.1, random_state=0):
    np.random.seed(random_state)
    _dataset = np.array(dataset)
    np.random.shuffle(_dataset)
    
    threshold = int(_dataset.shape[0] * 0.1)
    X_test = _dataset[:threshold, :-1]
    Y_test = _dataset[:threshold, -1]
    X_train = _dataset[threshold:, :-1]
    Y_train = _dataset[threshold:, -1]
    
    return X_train, X_test, Y_train, Y_test


X_train, X_test, Y_train, Y_test = train_test_split(entity.drop('movieId', axis=1), test_size=0.1, random_state=7)

In [103]:
%%time
clf = KNN(features=X_train, labels=Y_train, k=3)
preds = clf.predict(test_set=X_test)
print('Our KNN model Accuracy: {}'.format(clf.accuracy(Y_test, preds)))

Our KNN model Accuracy: 0.02049457994579946
CPU times: user 27 s, sys: 203 ms, total: 27.2 s
Wall time: 27.4 s


In [104]:
entity

,title,movieId,genres,rating
0,0,208297,1399,3.000000
1,1,51372,1599,2.896648
2,2,136604,1190,2.111111
3,3,183901,1571,3.750000
4,4,195955,1398,3.666667
...,...,...,...,...
59042,62320,159678,1524,3.250000
59043,62321,132604,1287,3.250000
59044,62322,132319,891,3.166667
59045,62323,130640,1571,2.375000
